# Tools and Agents in Strands

**Building Intelligent AI Agents with Custom Tools**

---

Welcome to this comprehensive tutorial on **tools and agents** in the Strands framework! This notebook provides a hands-on introduction to creating AI agents that can perform complex tasks by integrating with external tools and services. By the end of this 10-minute tutorial, you'll understand how to build powerful, tool-equipped agents.

### 🎯 What You'll Learn

In this tutorial, you will:
- Create agents with custom behaviors and personalities
- Build custom tools using the `@tool` decorator
- Understand type hints and their importance for tools
- Implement conversation context and memory
- Monitor agent performance and handle errors
- Create production-ready tool-equipped agents

### 🛠️ What Are Tools?

**Tools** in Strands are functions that agents can use to:
- Perform calculations or data processing
- Access external APIs and services
- Interact with databases or file systems
- Execute any custom logic you define

Think of tools as superpowers you give to your agents!

## 📦 Step 1: Installing Required Packages

### Overview
Let's start by installing the necessary packages for this tutorial.

### 📚 Packages We'll Install
- **strands-agents**: Core framework for building agents
- **strands-agents-tools**: Additional tool utilities
- **strands-agents-builder**: Agent configuration helpers

In [1]:
# Install Strands Agents SDK and related packages
%pip install strands-agents strands-agents-tools strands-agents-builder boto3 -q

print("✅ All packages installed successfully!")
print("   Ready to build intelligent agents with tools! 🚀")

Note: you may need to restart the kernel to use updated packages.
✅ All packages installed successfully!
   Ready to build intelligent agents with tools! 🚀


## 🔐 Step 2: Setting Up AWS Authentication

### Overview
Strands Agents SDK supports multiple model providers. We'll use AWS Bedrock to access Claude, one of the most powerful language models available.

### 🔑 Authentication Options
1. **AWS Bedrock API Keys** (EASIEST - Recommended for beginners)
2. **AWS Profile** (Traditional method)
3. **Environment Variables**
4. **IAM Roles** (Recommended for production)

### 📋 Required Permissions
Ensure your AWS credentials have:
- `bedrock:InvokeModel`
- `bedrock:InvokeModelWithResponseStream`

In [2]:
# 🚀 EASIEST OPTION: Paste your Bedrock API Key here
# Get your API key from: https://console.aws.amazon.com/bedrock/ -> API keys
API_KEY = ""  # Paste your API key between the quotes

# Configuration constants (you can modify these if needed)
REGION_NAME = "us-west-2"
AWS_PROFILE = "default"

# Set up authentication using our utility function
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent / "src"))  # Add the src directory to path for imports
from auth_utils import setup_bedrock_auth, display_auth_status
from strands import Agent

auth_status = setup_bedrock_auth(
    api_key=API_KEY,
    region_name=REGION_NAME,
    aws_profile=AWS_PROFILE
)

# Display the authentication status
display_auth_status(auth_status)

🔄 No API key provided, using traditional AWS authentication
   Checking for AWS credentials...
✅ AWS credentials found!

✅ Authentication Method: AWS Profile
   Using: AWS credentials file
   Region: us-west-2 (Profile: default)
💡 Ready to create agents!


## 🎨 Step 3: Creating Agents with Custom Personalities (Repetition)
In the previous notebook, we have learned how to create a basic agent that only passes input to the LLM. We also have augmented the behavior of the agent using system prompts. Let's repeat that briefly before diving into the topic of tools.

### System Prompts
System prompts define an agent's personality, expertise, and behavior. Let's create specialized agents for different purposes.

In [3]:
# Create a Python expert agent
python_expert = Agent(
    system_prompt="""You are a senior Python developer with 15 years of experience.
    You specialize in clean code, best practices, and teaching others.
    Always provide clear explanations and practical examples.""",
    model=auth_status.strands_bedrock_model
)

# Test the Python expert
question = "How do I create a list comprehension in Python?"
response = python_expert(question)

# Python List Comprehension

A list comprehension is a concise way to create lists in Python. It's one of Python's most distinctive features and can make your code more readable and efficient.

## Basic Syntax

The basic syntax is:

```python
new_list = [expression for item in iterable if condition]
```

Where:
- `expression` is what you want to include in the list
- `item` is the variable representing each element in the iterable
- `iterable` is any Python iterable (list, tuple, string, etc.)
- `condition` is optional and filters items (only include items that satisfy the condition)

## Simple Examples

### Creating a list of squares:
```python
squares = [x**2 for x in range(10)]
# Result: [0, 1, 4, 9, 16, 25, 36, 49, 64, 81]
```

### Filtering even numbers:
```python
even_numbers = [x for x in range(20) if x % 2 == 0]
# Result: [0, 2, 4, 6, 8, 10, 12, 14, 16, 18]
```

### Transforming strings:
```python
names = ['Alice', 'Bob', 'Charlie']
upper_names = [name.upper() for name in names

## 🔧 Step 4: Building Custom Tools

### The Power of Tools
Tools transform agents from conversational AI into action-oriented assistants. Let's create some useful tools!

### 🎯 Key Concepts
1. **@tool Decorator**: Marks a function as a tool
2. **Type Hints**: Critical for proper parameter conversion
3. **Docstrings**: Help the agent understand when to use each tool

In [9]:
from strands import tool
import datetime
import random
import math

# Tool 1: Get current time
@tool
def get_current_time():
    """Get the current date and time."""
    return datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")

# Tool 2: Generate random numbers
@tool
def generate_random_number(min_val: int = 1, max_val: int = 100):
    """Generate a random number between min_val and max_val.
    
    Args:
        min_val: Minimum value (default: 1)
        max_val: Maximum value (default: 100)
    """
    return random.randint(min_val, max_val)

# Tool 3: Calculate factorial
@tool
def calculate_factorial(n: int):  # Note: Paarameter has no "type hint" => str assumed
    """Calculate the factorial of a number.
    
    Args:
        n: The number to calculate factorial for
    """
    if n < 0:
        return "Error: Factorial is not defined for negative numbers"
    elif n == 0 or n == 1:
        return 1
    else:
        return math.factorial(n)

# Tool 4: Calculate compound interest
@tool
def calculate_compound_interest(principal: float, rate: float, time: int, compounds_per_year: int = 12):
    """Calculate compound interest.
    
    Args:
        principal: Initial amount
        rate: Annual interest rate (as a percentage)
        time: Time period in years
        compounds_per_year: Number of times interest compounds per year (default: 12)
    """
    rate_decimal = rate / 100
    amount = principal * (1 + rate_decimal/compounds_per_year) ** (compounds_per_year * time)
    interest = amount - principal
    return {
        "principal": principal,
        "final_amount": round(amount, 2),
        "interest_earned": round(interest, 2),
        "rate": rate,
        "time": time
    }

print("🔧 Custom tools created!")
print("   1. get_current_time - Returns current date/time")
print("   2. generate_random_number - Random number generator")
print("   3. calculate_factorial - Mathematical calculations")
print("   4. calculate_compound_interest - Financial calculations")

🔧 Custom tools created!
   1. get_current_time - Returns current date/time
   2. generate_random_number - Random number generator
   3. calculate_factorial - Mathematical calculations
   4. calculate_compound_interest - Financial calculations


## 🤖 Step 5: Creating a Tool-Equipped Agent

### Bringing It All Together
Now let's create an agent that can use all our custom tools. This agent will be able to perform calculations, generate random numbers, and more!

In [10]:
# Create a tool-equipped agent
tool_agent = Agent(
    system_prompt="""You are a helpful assistant equipped with various tools.
    
    🛠️ YOUR CAPABILITIES:
    - Time and date information
    - Random number generation
    - Mathematical calculations (factorial)
    - Financial calculations (compound interest)
    
    📋 GUIDELINES:
    - Use tools when appropriate to provide accurate information
    - Explain your calculations and results clearly
    - If a user's request is unclear, ask for clarification
    - Be friendly and helpful in all interactions""",
    tools=[
        get_current_time,
        generate_random_number,
        calculate_factorial,
        calculate_compound_interest
    ],
    model=auth_status.strands_bedrock_model
)

print("🤖 Tool-equipped agent created!")
print("   Personality: Helpful assistant")
print("   Tools: 4 custom functions")
print("   Ready to help with calculations and more!")

🤖 Tool-equipped agent created!
   Personality: Helpful assistant
   Tools: 4 custom functions
   Ready to help with calculations and more!


## 🎯 Step 6: Testing Our Tool-Equipped Agent

### Usage Examples
Let's test our agent with various tasks that require tool usage. Watch how it intelligently decides which tools to use!

In [12]:
# Test various tool capabilities
test_questions = [
    "What time is it right now?",
    "Generate a random number",     # default values are used!
    "Generate 3 random numbers between 100 and 200",  # explicit interval with min and max
    "What's the factorial of 7?",
    "If I invest $10,000 at 5% annual interest with monthly compound for 10 years, how much will I have?",
]

print("🎯 Testing tool-equipped agent...\n")

for i, question in enumerate(test_questions, 1):
    print(f"{'='*60}")
    print(f"Question {i}: {question}")
    
    response = tool_agent(question)
    print(f"\n{'='*60}\n\n")

🎯 Testing tool-equipped agent...

Question 1: What time is it right now?
I'll check the current time for you.
Tool #18: get_current_time
The current time is July 20, 2025, at 11:08:34 AM.


Question 2: Generate a random number
I'll generate a random number for you using the default range of 1 to 100.
Tool #19: generate_random_number
Your random number is 76.

If you'd like a random number within a different range, just let me know the minimum and maximum values you'd prefer.


Question 3: Generate 3 random numbers between 100 and 200
I'll generate 3 random numbers for you, each between 100 and 200.
Tool #20: generate_random_number

Tool #21: generate_random_number

Tool #22: generate_random_number
Here are your 3 random numbers between 100 and 200:
1. 133
2. 128
3. 170


Question 4: What's the factorial of 7?
I'll calculate the factorial of 7 for you.
Tool #23: calculate_factorial
The factorial of 7 is 5,040.

To explain: The factorial of a number (denoted as n!) is the product of all 

⚠️ **IMPORTANT**: While modern LLMs will be able to answer "What's the factorial of 7" even without a tool, try passing "If I invest $10,000 at 5% annual interest with monthly compound for 10 years, how much will I have?" to an LLM directly without tools. Particularly smaller models will provide you with answers that are almost right, but just not right. Hence, remember that LLMs are not good with math & logic, this is one of the reasons why you need tools.

## 💬 Step 7: Combining tools and conversation Context
In the previous notebook, we also showed that Strands keeps context, i.e., it will use previous prompts and answers as a context for building the next answer. We can combine this concept with tools now.

### Memory and Context
Strands agents maintain conversation history, allowing for natural, contextual interactions. Let's see this in action!

In [13]:
# Create a fresh agent for conversation demo
conversation_agent = Agent(
    system_prompt="You are a friendly math tutor. Help students understand concepts step by step.",
    tools=[calculate_factorial, generate_random_number],
    model=auth_status.strands_bedrock_model
)

print("💬 Starting a contextual conversation...\n")

# Conversation flow
conversation = [
    "Hi! I'm learning about factorials.",
    "Can you calculate 5 factorial for me?",
    "How is that different from 6 factorial?",
    "Can you show me a pattern with factorials from 1 to 5?"
]

for turn, message in enumerate(conversation, 1):
    print(f"\n{'='*60}")
    print(f"Turn {turn} - You: {message}\n")
    
    response = conversation_agent(message)
    print(f"{'='*60}")

💬 Starting a contextual conversation...


Turn 1 - You: Hi! I'm learning about factorials.

Hi there! I'd be happy to help you learn about factorials. 

Factorials are an important concept in mathematics. The factorial of a non-negative integer n, denoted by n!, is the product of all positive integers less than or equal to n.

For example:
- 0! = 1 (by definition)
- 1! = 1
- 2! = 2 × 1 = 2
- 3! = 3 × 2 × 1 = 6
- 4! = 4 × 3 × 2 × 1 = 24
- And so on...

Would you like me to calculate a specific factorial for you? Or do you have any particular questions about factorials that I can help with?============================================================

Turn 2 - You: Can you calculate 5 factorial for me?

I'd be happy to calculate 5 factorial (5!) for you!
Tool #25: calculate_factorial
5! = 120

Let me break this down step by step:
5! = 5 × 4 × 3 × 2 × 1 = 120

The calculation works by multiplying all integers from 5 down to 1:
- Start with 5
- Multiply by 4 to get 20
- Multiply by 3 to get

## ⚠️ Step 8: Error Handling and Edge Cases

### Building Robust Agents
Good agents handle errors gracefully. Let's test how our agent deals with invalid inputs and edge cases.

In [14]:
# Test error handling
error_test_cases = [
    "Calculate the factorial of -5",
    "Generate a random number between 100 and 50",  # Min > Max
    "What's the compound interest on negative $1000?",
    ""  # Empty input
]

print("⚠️ Testing error handling...\n")

for i, test_case in enumerate(error_test_cases, 1):
    print(f"Test {i}: '{test_case}'")
    try:
        response = tool_agent(test_case) if test_case else tool_agent(" ")
    except Exception as e:
        print(f"❌ Error caught: {e}")
    print(f"\n{'=' * 40}\n\n")

print("\n✅ Error handling test completed!")

⚠️ Testing error handling...

Test 1: 'Calculate the factorial of -5'
I apologize, but I can't calculate the factorial of a negative number like -5. In mathematics, factorial is only defined for non-negative integers (0 and positive whole numbers).

The factorial function (n!) represents the product of all positive integers less than or equal to n:
n! = n × (n-1) × (n-2) × ... × 2 × 1

For negative numbers like -5, the factorial is not mathematically defined in the standard factorial function.

If you'd like, I can calculate the factorial of a positive number instead. Would you like me to do that?


Test 2: 'Generate a random number between 100 and 50'
There seems to be a bit of confusion in your request. For generating a random number, the minimum value should be less than the maximum value. You've asked for a random number between 100 and 50, but 100 is greater than 50.

Did you mean to generate a random number between 50 and 100 instead? If so, I'd be happy to do that for you. Other

## 🎓 Build yourself: Creating a Multi-Tool Research Assistant

### Combining Multiple Capabilities
Let's check if you can write your own tools for complex tasks and more advanced agents.

In [ ]:
# Additional tools for research
@tool
def word_count(text: str):
    """Count the number of words in a text."""
    # YOU NEED TO EDIT THE CODE HERE
    pass

@tool
def calculate_reading_time(word_count: int, words_per_minute: int = 200):
    """Calculate estimated reading time in minutes.
    
    Args:
        word_count: Number of words in the text
        words_per_minute: Reading speed (default: 200)
    """
    # YOU NEED TO EDIT THE CODE HERE
    pass

# Create research assistant
research_assistant = Agent(
    system_prompt="""You are an advanced research assistant capable of complex analysis.
    
    Your capabilities include:
    - Mathematical calculations
    - Text analysis
    - Time estimations
    - Data synthesis
    
    Always provide comprehensive, well-structured responses.""",
    tools=[
        calculate_factorial,
        calculate_compound_interest,
        word_count,
        calculate_reading_time,
        get_current_time,
        generate_random_number
    ],
    model=auth_status.strands_bedrock_model
)

# Complex research task
research_task = """I'm writing a 5000-word research paper about compound interest. 
Can you:
1. Calculate how long it will take to read
2. Show me how $1000 grows over 20 years at 7% interest
3. Give me 5 random page numbers between 1-50 for citations
"""

print("🎓 Testing Research Assistant")
print("=" * 80)
print(f"Task: {research_task}")
print("=" * 80)

response = research_assistant(research_task)

## 🎉 Congratulations!

### 🏆 What You've Accomplished
In this tutorial, you've learned how to:
- ✅ Create basic and specialized agents
- ✅ Build custom tools with the `@tool` decorator
- ✅ Use type hints for reliable tool execution
- ✅ Implement conversation context and memory
- ✅ Handle errors gracefully
- ✅ Monitor and optimize performance
- ✅ Combine multiple tools for complex tasks

### 🚀 Key Takeaways

1. **Tools = Superpowers**: Tools transform conversational agents into action-oriented assistants
2. **Type Hints Matter**: Proper type hints ensure reliable tool execution
3. **Context is King**: Agents maintain conversation history for natural interactions
4. **Error Handling**: Robust agents handle edge cases gracefully

### 💡 Best Practices

1. **Clear Docstrings**: Help agents understand when to use each tool
2. **Type Annotations**: Always include type hints for parameters
3. **Error Handling**: Handle edge cases within your tools
4. **System Prompts**: Design clear instructions for agent behavior

### 🔮 What's Next?

Now that you've mastered tools and agents, you're ready to:
1. **Build Complex Tools**: Integrate with APIs, databases, and external services
2. **Create Specialized Agents**: Domain-specific assistants for your needs
3. **Implement Workflows**: Chain multiple agents and tools together
4. **Deploy to Production**: Build real-world applications

### 📚 Resources

- [Strands Documentation](https://strandsagents.com/0.1.x/)
- [AWS Bedrock Documentation](https://docs.aws.amazon.com/bedrock/)
- [AWS Bedrock API Keys Guide](https://docs.aws.amazon.com/bedrock/latest/userguide/api-keys.html)
- [Tool Development Guide](https://strandsagents.com/0.1.x/user-guide/concepts/tools/)

### 🌟 Challenge Yourself

Try creating:
- A weather tool that fetches current conditions
- A database query tool for data analysis
- An email sending tool for notifications
- A file processing tool for document analysis

The possibilities are endless with Strands Agents!

Happy building! 🚀🤖✨